In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np

In [3]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoint_with_scores = outputs['output_0'].numpy()
    return keypoint_with_scores

In [4]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [5]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[19:]
    
  
  
    print(name)
    print(subname)
    if (name.find("bhuj") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    else:
        save_path = os.path.join(save_dir, "Tadasana",subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            image_tensor = tf.convert_to_tensor(np.array(image))
            input_tensor = tf.image.resize_with_pad(image_tensor, input_size, input_size)
            input_tensor = tf.expand_dims(input_tensor, axis=1)

            processed_image_array = []
            keypoint_array = []

            for image in input_tensor:
              # Run model inference.
              keypoint_with_scores = movenet(image)

              #print(keypoint_with_scores.shape)

              # Visualize the predictions with image.
              display_image = image
              display_image = tf.cast(tf.image.resize_with_pad(
                  display_image, 1280, 1280), dtype=tf.int32)
              output_overlay = draw_prediction_on_image(
                  np.squeeze(display_image.numpy(), axis=0), keypoint_with_scores)

              processed_image_array.append(output_overlay)
              # keypoint_array = keypoint_with_scores ->

            cv2.imwrite(f"{save_path}/{idx}.png", output_overlay)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("C:/BE_project/Yoga_Vid_Collected/*")
    save_dir = "dataset1"

    for path in video_paths:
        save_frame(path, save_dir, gap=30)

yoga_vid_collected\abhay_bhuj
abhay_bhuj
dataset1\Bhujangasana\abhay_bhuj


InvalidArgumentError:  input must be 4-dimensional[1,192,3]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/model/Conv1_relu/Relu6}}]] [Op:__inference_signature_wrapper_39225]